In [1]:
import pandas as pd
samples_path = '../logs/samples.csv'
samples = pd.read_csv(samples_path, index_col=0)
samples = [(s[0:25].tolist(), s[25:].tolist()) for s in samples.values][0:1]

In [2]:
from sensitivity import test_sensitivity

nnet_path = '../network/models/v3.2.0/model.nnet'
sym_results = test_sensitivity(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/sensitivity/symmetric')
asym_results = test_sensitivity(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, asym=True, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/sensitivity/asymmetric')

print('Symmetric results: ', sym_results)
print('Asymmetric results: ', asym_results)

INFO:sensitivity:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:sensitivity:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:sensitivity:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.01 and 0.09
INFO:sensitivity:searching 9 coarse epsilons b/t 0.1 and 0.9
INFO:sensitivity:searching 9 coarse epsilons b/t 1.0 and 9.0
INFO:sensitivity:x0_s0 coarse epsilon bounds: (4.0, 5.0) (2930ms)
INFO:sensitivity:searching 100000 epsilons b/t 4.0 and 4.99999
INFO:sensitivity:x0_s0 epsilon: 4.13357 (1551ms)
INFO:sensitivity:x0 epsilon: (-4.13357, 4.13357) (4483ms)
INFO:sensitivity:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:sensitivity:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:sensitivity:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.01 a

In [3]:
from sensitivity import check_sensitivity

sym_check_results = check_sensitivity(nnet_path, samples, sym_results)
asym_check_results = check_sensitivity(nnet_path, samples, asym_results, asym=True)

print('Symmetric check results:', sym_check_results)
print('Asymmetric check results:', asym_check_results)

INFO:sensitivity:x0 ok (-4.13357, 4.13357)
INFO:sensitivity:x1 ok (-0.0606, 0.0606)
INFO:sensitivity:x2 ok (-0.05716, 0.05716)
INFO:sensitivity:x3 ok (-0.89633, 0.89633)
INFO:sensitivity:x4 ok (-0.97788, 0.97788)
INFO:sensitivity:x5 ok (-0.60006, 0.60006)
INFO:sensitivity:x6 ok (-0.48063, 0.48063)
INFO:sensitivity:x7 ok (-0.94469, 0.94469)
INFO:sensitivity:x8 ok (-0.30256, 0.30256)
INFO:sensitivity:x9 ok (-0.30484, 0.30484)
INFO:sensitivity:x10 ok (-0.5447, 0.5447)
INFO:sensitivity:x11 ok (-1.04818, 1.04818)
INFO:sensitivity:x12 ok (-1.25774, 1.25774)
INFO:sensitivity:x13 ok (-0.55866, 0.55866)
INFO:sensitivity:x14 ok (-1.22048, 1.22048)
INFO:sensitivity:x15 ok (-2.3449, 2.3449)
INFO:sensitivity:x16 ok (-0.30297, 0.30297)
INFO:sensitivity:x17 ok (-0.36035, 0.36035)
INFO:sensitivity:x18 ok (-0.22711, 0.22711)
INFO:sensitivity:x19 ok (-0.07387, 0.07387)
INFO:sensitivity:x20 ok (-1.90717, 1.90717)
INFO:sensitivity:x21 ok (-0.61339, 0.61339)
INFO:sensitivity:x22 ok (-0.8306, 0.8306)
INFO:s